In [36]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [86]:
%matplotlib inline
import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

import pandas as pd
import seaborn as sns
import numpy as np

import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split, KFold
from sklearn.grid_search import GridSearchCV

import transform_utils as tu
from kaggle_helper import KaggleHelper

In [91]:
kh = KaggleHelper("submissions.db", "predictions")

table submissions already exists
table validation_scores already exists


In [88]:
%aimport transform_utils
%aimport kaggle_helper

In [89]:
%aimport

Modules to reload:
kaggle_helper transform_utils

Modules to skip:



In [90]:
%autoreload

In [69]:
pd.read_csv("data/train.csv").head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,999,5,Friday,68113152929,-1,FINANCIAL SERVICES,1000
1,30,7,Friday,60538815980,1,SHOES,8931
2,30,7,Friday,7410811099,1,PERSONAL CARE,4504
3,26,8,Friday,2238403510,2,PAINT AND ACCESSORIES,3565
4,26,8,Friday,2006613744,2,PAINT AND ACCESSORIES,1017


In [70]:
df = tu.preprocess_walmart(pd.read_csv("data/train.csv"))
df2 = tu.transform_column(df, 'DepartmentDescription', ['TripType', 'VisitNumber'])
df3 = tu.transform_group(df2, 'VisitNumber', True)

In [71]:
X = df3.drop("TripType", axis=1).drop("VisitNumber", axis=1)
y = df3.TripType

In [72]:
a_train, a_test, b_train, b_test = train_test_split(X.values, y.values, test_size=0.2)

http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html#sklearn.grid_search.GridSearchCV

In [73]:
# for x in [50, 75, 100]:
#     for train, test in KFold(len(a_train), 3):
#         clf = RandomForestClassifier(n_estimators=250, n_jobs=3, max_depth=x)
#         clf.fit(a_train[train], b_train[train])
#         print("max depth:", x, clf.score(a_train[test], b_train[test]))

In [92]:
clf = RandomForestClassifier(n_jobs=3)
clf.fit(a_train, b_train)
score = clf.score(a_test, b_test)
kh.save_validation_score(clf, score, len(a_train), "")

In [54]:
test = tu.preprocess_walmart(pd.read_csv("data/test.csv"))
test2 = tu.transform_column(test, 'DepartmentDescription', ['VisitNumber'])
test3 = tu.transform_group(test2, 'VisitNumber')

In [55]:
test_predictions = clf.predict(test3.values)
output = tu.convert_predictions(test_predictions, 
                               output_index=pd.Series(test3.index), 
                               actual_trip_types=df.TripType.unique())

Missing Trip Types: ['TripType_14']


In [62]:
kh.save_predictions(output, clf, len(a_train), "")